In [ ]:
from openhunt.logparser import winlogbeat
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
win = winlogbeat()
spark = SparkSession.builder.appName("Process").config("spark.sql.caseSensitive", "True").getOrCreate()

In [ ]:
process_injection = win.extract_nested_fields("empire_psinject_2019-05-18200432.json",spark)

In [ ]:
DLL_Injection = spark.sql(
'''
SELECT 
    ltrim('0x', a.GrantedAccess) as granted_access,
    b.ImageLoaded,
    a.GrantedAccess,
    b.Image,
    a.SourceProcessId,
    b.ProcessId
FROM process_injection b
JOIN process_injection a
    ON a.TargetProcessGuid = b.ProcessGuid
    AND a.event_id = 10
    
    AND b.Image = a.TargetImage
    AND b.channel = "Microsoft-Windows-Sysmon/Operational"
JOIN process_injection c
    ON b.Image = c.TargetImage
    AND c.event_id = 8
    AND c.StartAddress = "LoadLibraryA"
    AND b.channel = "Microsoft-Windows-Sysmon/Operational"
WHERE b.channel = "Microsoft-Windows-Sysmon/Operational"
    AND b.event_id = 7
'''
)

In [ ]:
df = DLL_Injection.select("computer_name", conv(DLL_Injection.granted_access, 16, 10).alias("granted_access"), "SourceImage", "SourceProcessGuid", "NewThreadId", "TargetProcessId", "TargetImage")

In [ ]:
df.createOrReplaceTempView("df")

In [ ]:
eid_7_8_10_minimal_privileges = spark.sql(
'''
SELECT * FROM df WHERE ((INT(granted_access) & 40) == 40) -- 40 is decimal for 0x0028. The minimal privileges you need to access process handle
'''
).show(1,False)